In [38]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
import chart_studio
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objects as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from chart_studio.plotly import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')


In [36]:
!pip install cufflinks

distutils: D:\Users\PiusHwang\anaconda3\Include\UNKNOWN
sysconfig: D:\Users\PiusHwang\anaconda3\Include
user = False
home = None
root = None
prefix = None
distutils: D:\Users\PiusHwang\anaconda3\Include\UNKNOWN
sysconfig: D:\Users\PiusHwang\anaconda3\Include
user = False
home = None
root = None
prefix = None


  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=68724 sha256=15ac62cd991409c2b402bb2ce50c1bb5d60c2b8067590958dfb9e51adf56f9c6
  Stored in directory: c:\users\piushwang\appdata\local\pip\cache\wheels\e1\27\13\3fe67fa7ea7be444b831d117220b3b586b872c9acd4df480d0
Successfully built cufflinks


In [33]:
chart_studio.plotly.gra

<module 'chart_studio.plotly.plotly' from 'D:\\Users\\PiusHwang\\anaconda3\\lib\\site-packages\\chart_studio\\plotly\\plotly.py'>

In [3]:
df = pd.read_csv('complaints.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2335289 entries, 0 to 2335288
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company public response       object
 7   Company                       object
 8   State                         object
 9   ZIP code                      object
 10  Tags                          object
 11  Consumer consent provided?    object
 12  Submitted via                 object
 13  Date sent to company          object
 14  Company response to consumer  object
 15  Timely response?              object
 16  Consumer disputed?            object
 17  Complaint ID                  int64 
dtypes: int64(1), object(17)
memory usage: 320.

In [5]:
df.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    866822
Debt collection                                                                 389432
Mortgage                                                                        340515
Credit reporting                                                                140431
Credit card or prepaid card                                                     125513
Checking or savings account                                                     104160
Credit card                                                                      89190
Bank account or service                                                          86206
Student loan                                                                     63651
Money transfer, virtual currency, or money service                               34011
Consumer Loan                                                                    31600
Vehicle loan or lease                      

In [6]:
df_HADOOP = pd.read_csv('HADOOP_aug2_word.csv')

In [7]:
df_HADOOP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12304 entries, 0 to 12303
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12304 non-null  object
 1   labels  12304 non-null  object
dtypes: object(2)
memory usage: 192.4+ KB


In [8]:
df.loc[df['Product'] == 'Credit reporting', 'Product'] = 'Credit reporting, credit repair services, or other personal consumer reports'
df.loc[df['Product'] == 'Credit card', 'Product'] = 'Credit card or prepaid card'
df.loc[df['Product'] == 'Payday loan', 'Product'] = 'Payday loan, title loan, or personal loan'
df.loc[df['Product'] == 'Virtual currency', 'Product'] = 'Money transfer, virtual currency, or money service'
df = df[df.Product != 'Other financial service']

In [18]:
df['Product'].value_counts().sort_values(ascending=False).iplot(kind = 'bar', yTitle = 'Number of Complaints', title = 'Number complaints in each product')

AttributeError: 'Series' object has no attribute 'iplot'

In [17]:
!pip install -U ipykernel

distutils: D:\Users\PiusHwang\anaconda3\Include\UNKNOWN
sysconfig: D:\Users\PiusHwang\anaconda3\Include
user = False
home = None
root = None
prefix = None
distutils: D:\Users\PiusHwang\anaconda3\Include\UNKNOWN
sysconfig: D:\Users\PiusHwang\anaconda3\Include
user = False
home = None
root = None
prefix = None


In [16]:
print(df.loc[1])

Date received                                                          2019-11-01
Product                                                     Vehicle loan or lease
Sub-product                                                                  Loan
Issue                                                 Struggling to pay your loan
Sub-issue                                        Denied request to lower payments
Consumer complaint narrative    I contacted Ally on Friday XX/XX/XXXX after fa...
Company public response         Company has responded to the consumer and the ...
Company                                                       ALLY FINANCIAL INC.
State                                                                          NJ
ZIP code                                                                    088XX
Tags                                                                          NaN
Consumer consent provided?                                       Consent provided
Submitted via   

In [15]:
def print_plot(index):
    example = df[df.index == index][['Consumer complaint narrative', 'Product']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Product:', example[1])
print_plot(1)

I contacted Ally on Friday XX/XX/XXXX after falling behind on payments due to being out of work for a short period of time due to an illness. I chated with a representative after logging into my account regarding my opitions to ensure I protect my credit and bring my account current. 

She advised me that before an extenstion could be done, I had to make a payment in the amount of {$270.00}. I reviewed my finances, as I am playing catch up on all my bills and made this payment on Monday XX/XX/XXXX. This rep advised me, once this payment posts to my account to contact Ally back for an extention or to have a payment deffered to the end of my loan. 

With this in mind, I contacted Ally again today and chatted with XXXX. I explained all of the above and the information I was provided when I chatted with the rep last week. She asked several questions and advised me that a one or two month  extension/deffered payment could be done however partial payment is needed! WHAT? She advised me {$230

In [40]:
df = df.reset_index(drop=True)
df = df.astype(str)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))


def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(clean_text)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].str.replace('\d+', '')

D:\Users\PiusHwang\anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning:

The default value of regex will change from True to False in a future version.



In [41]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Consumer complaint narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 208792 unique tokens.
